In [2]:
pip install pomegranate==v0.14.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for pomegranate: filename=pomegranate-0.14.9-cp310-cp310-linux_x86_64.whl size=18331316 sha256=05f53ad1a7462a2b30b7fc38344c2efeb0a300e8dad4dd07355a7b4642b13294
  Stored in directory: /root/.cache/pip/wheels/14/e7/b2/189a2d351ac4ae073cfa17ce9d56936d59af5712a18028fc31
Successfully built pomegranate
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.4
    Uninstalling Cython-3.0.4:
      Successfully uninstalled Cython-3.0.4


In [10]:
import math
from pomegranate import *

burglary = DiscreteDistribution({'1': 0.001, '0': 0.999})
earthquake = DiscreteDistribution({'1': 0.002, '0': 0.998})

In [11]:
alarm = ConditionalProbabilityTable(
        [[ '0', '0', '1', 0.001 ],
         [ '0', '1', '1', 0.29 ],
         [ '1', '0', '1', 0.94 ],
         [ '1', '1', '1', 0.95 ],
         [ '0', '0', '0', 0.999 ],
         [ '0', '1', '0', 0.71 ],
         [ '1', '0', '0', 0.06 ],
         [ '1', '1', '0', 0.05 ]],[burglary,earthquake])
johncalls = ConditionalProbabilityTable(
        [[ '0', '0', 0.95 ],
         [ '0', '1', 0.05 ],
         [ '1', '0', 0.10 ],
         [ '1', '1', 0.90 ]],[alarm])
marrycalls = ConditionalProbabilityTable(
        [[ '0', '0', 0.99 ],
         [ '0', '1', 0.01 ],
         [ '1', '0', 0.30 ],
         [ '1', '1', 0.70 ]],[alarm])

In [12]:
Burglary = State(burglary, name='Burglary')
Earthquake = State(earthquake, name='Earthquake')
Alarm = State(alarm, name='Alarm')
JohnCalls = State(johncalls, name='JohnCalls')
MarryCalls = State(marrycalls, name='MarryCalls')

In [13]:
model = BayesianNetwork('Assignment1')
model.add_states(Burglary,Earthquake,Alarm,JohnCalls,MarryCalls)

model.add_edge(Burglary,Alarm)
model.add_edge(Earthquake,Alarm)
model.add_edge(Alarm,JohnCalls)
model.add_edge(Alarm,MarryCalls)

model.bake()

In [22]:
# P(JC|A)
pja = model.predict_proba([{'Alarm':'1'}])[0][3].parameters[0]['1']
pja

0.8999999999999998

In [24]:
# P(A|~B,~E)
panbe = model.predict_proba([{'Burglary':'0','Earthquake':'0'}])[0][2].parameters[0]['1']
panbe

0.0010000000000006649

In [25]:
# P(MC|A)
pma = model.predict_proba([{'Alarm':'1'}])[0][4].parameters[0]['1']
pma

0.6999999999999998

In [35]:
# P(JC,MC,A,~B,~E) = P(JC|A)*P(MC|A)*P(A|~B,~E)*P(~B)*P(~E)
pnb = model.predict_proba([{'Burglary':'0'}])[0][1].parameters[0]['0']
pne = model.predict_proba([{'Earthquake':'0'}])[0][0].parameters[0]['0']
ans = pja*pma*panbe*pnb*pne
ans

0.0006281112600004167